OpenNMT gives us a way to use beam search, which could be tricky to implement efficiently on your own.

Configure different models for OpenNMT including Transformer and included sentence piece model.

OpenNMT, is similar to other ML frameworks in that it relies on a combination of editable .yaml files and command line tools to run the training procedure.  




In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0 MB 10 kB/s 
     |████████████████████████████████| 5.9 MB 46.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0+cu101 which is incompatible.


# Cloning OpenNMT

In [ ]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git
!cd OpenNMT-py; pip install -e .
# !wget https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
# !tar xf toy-ende.tar.gz


Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17675, done.
remote: Counting objects: 100% (631/631), done.
remote: Compressing objects: 100% (350/350), done.
remote: Total 17675 (delta 373), reused 463 (delta 271), pack-reused 17044
Receiving objects: 100% (17675/17675), 273.81 MiB | 24.26 MiB/s, done.
Resolving deltas: 100% (12673/12673), done.
Obtaining file:///content/OpenNMT-py
     |████████████████████████████████| 73 kB 1.3 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 16.6 MB 76 kB/s 
     |████████████████████████████████| 1.2 MB 37.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
  Running setup.py develop for OpenNMT-py



### Build the vocab for the Multi30k En-Fr dataset

This relies on having a config file with the below information laid out.

While just having a vocabulary is fine for some cases, using a sub-word tokenization might help capture morphological information better.

To do this, we add ```transforms: [sentencepiece, filtertoolong]``` to both the training and validation corpora in the config file yml.


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import spacy
# spacy_en = spacy.load('en_core_web_sm')
# # punctuation = set(string.punctuation)
# # stop_words = set(stopwords.words('english'))
# def tokenize_en(text):
#     """
#     Tokenizes English text from a string into a list of strings (tokens)
#     """
#     token_list =[]
#     for tok in spacy_en.tokenizer(text):
#       token_list.append(tok.text)

#     return ' '.join(token_list)

In [ ]:
# import sentencepiece as spm
# spm.SentencePieceTrainer.train(input='/content/drive/MyDrive/Lab3/data/train_eng_tok.txt', model_prefix='en',vocab_size=6797)
# spm.SentencePieceTrainer.train(input='/content/drive/MyDrive/Lab3/data/train_fre_tok.txt', model_prefix='fr', vocab_size=6797)

In [ ]:
## TO DO COMPLETE DATA SAVING

# Corpus opts:

## TODO COMPLETE CORPUS OPTIONS
src_subword_model: ./fr.model
tgt_subword_model: ./en.model
src_subword_vocab: ./fr.vocab
tgt_subword_vocab: ./en.vocab
## Where the samples will be written
save_data: drive/MyDrive/Lab3/data/run/example
## Where the vocab(s) will be written
# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: ./drive/MyDrive/Lab3/data/train_eng.txt
        path_tgt: ./drive/MyDrive/Lab3/data/train_fre.txt
        transforms: [sentencepiece, filtertoolong]
    valid:
        path_src: ./drive/MyDrive/Lab3/data/val_eng.txt
        path_tgt: ./drive/MyDrive/Lab3/data/val_fre.txt
        transforms: [sentencepiece]



# Vocabulary files that were just created
src_vocab: drive/MyDrive/Lab3/data/run/example.vocab.src
tgt_vocab: drive/MyDrive/Lab3/data/run/example.vocab.tgt



In [ ]:
!onmt_build_vocab -config multi30k.yml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-11 05:59:43,906 INFO] Counter vocab from 10000 samples.
[2022-03-11 05:59:43,906 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-03-11 05:59:43,917 INFO] corpus_1's transforms: TransformPipe()
[2022-03-11 05:59:44,224 INFO] Counters src:9483
[2022-03-11 05:59:44,224 INFO] Counters tgt:8615



# Train Model

Next we will beging training with OpenNMT, again using the same config file, however, below we'll look at the relevant parts.
We fill in the multi30k.yaml config to setup a seq2seq model that has a 3 layer RNN encoder 2 layer RNN decoder, MLP attention, with 20% dropout, using Adam as our optimizer.

In [ ]:
## Should have 3 layers in encoder and 2 layers in decoder
## 20% dropout and 500 hidden units
# Model (note these are actually default values, but I've explicitely written them out to show how you can edit them)
decoder_type: rnn
encoder_type: rnn 
enc_layers: 3
dec_layers: 2
enc_rnn_size: 500
dec_rnn_size: 500
dropout: 0.2
global_attention : mlp
# Optimizer settings
optim: adam
learning_rate: 0.001

In [ ]:
!onmt_train -config multi30k.yml

[2022-03-11 05:59:57,841 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-11 05:59:57,842 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-11 05:59:57,842 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-11 05:59:57,845 INFO] Parsed 2 corpora from -data.
[2022-03-11 05:59:57,846 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-11 05:59:57,846 INFO] Loading vocab from text file...
[2022-03-11 05:59:57,846 INFO] Loading src vocabulary from drive/MyDrive/Lab3/data/run/example.vocab.src
[2022-03-11 05:59:57,872 INFO] Loaded src vocab has 9483 tokens.
[2022-03-11 05:59:57,877 INFO] Loading tgt vocabulary from drive/MyDrive/Lab3/data/run/example.vocab.tgt
[2022-03-11 05:59:57,897 INFO] Loaded tgt vocab has 8615 tokens.
[2022-03-11 05:59:57,901 INFO] Building fields with vocab in counters...
[2022-03-11 05:59:57,912 INFO]  * tgt vocab size: 8619.
[2022-03-11 05:59:57


# Decoding

Once our model is saved. We can use it to actually generate predictions on our output files. Our models will be saved under the ```save_model``` setting of our config file.

We create predictions for the validation set using your saved models and select the one that has the highest BLEU.

In [ ]:
## Code to create predictions and calculate BLEU for models

In [ ]:
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4000.pt -src drive/MyDrive/Lab3/data/val_fre.txt -output drive/MyDrive/Lab3/data/val_4000.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4500.pt -src drive/MyDrive/Lab3/data/val_fre.txt -output drive/MyDrive/Lab3/data/val_4500.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_5000.pt -src drive/MyDrive/Lab3/data/val_fre.txt -output drive/MyDrive/Lab3/data/val_5000.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2

[2022-03-11 06:37:43,982 INFO] Translating shard 0.
[2022-03-11 06:38:06,127 INFO] PRED AVG SCORE: -0.4101, PRED PPL: 1.5070
[2022-03-11 06:38:09,733 INFO] Translating shard 0.
[2022-03-11 06:38:32,359 INFO] PRED AVG SCORE: -0.3432, PRED PPL: 1.4094
[2022-03-11 06:38:35,987 INFO] Translating shard 0.
[2022-03-11 06:38:57,039 INFO] PRED AVG SCORE: -0.3328, PRED PPL: 1.3948


Finally let's calculate the BLEU scores of the outputs! We would eventually want to select the model with Highest BLEU

In [ ]:
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/val_eng.txt < drive/MyDrive/Lab3/data/val_4000.txt
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/val_eng.txt < drive/MyDrive/Lab3/data/val_4500.txt
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/val_eng.txt < drive/MyDrive/Lab3/data/val_5000.txt

BLEU = 35.79, 63.1/43.1/29.8/20.9 (BP=0.992, ratio=0.993, hyp_len=12076, ref_len=12167)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 36.45, 63.6/43.1/30.0/21.4 (BP=1.000, ratio=1.010, hyp_len=12287, ref_len=12167)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 35.87, 65.1/44.2/30.6/21.5 (BP=0.966, ratio=0

# Comparing Beam Width

For our BEST model we compare the peformance (Both BLEU and clocktime to run)  with the following Beam Sizes: 5 (done above), 10, 15, and 20.

In [ ]:
## TODO Beam comparison
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4500.pt -src drive/MyDrive/Lab3/data/val_fre.txt -output drive/MyDrive/Lab3/data/val_4500_5.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4500.pt -src drive/MyDrive/Lab3/data/val_fre.txt -output drive/MyDrive/Lab3/data/val_4500_10.txt -gpu 0 -beam_size 10 -seed 531 -block_ngram 2
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4500.pt -src drive/MyDrive/Lab3/data/val_fre.txt -output drive/MyDrive/Lab3/data/val_4500_15.txt -gpu 0 -beam_size 15 -seed 531 -block_ngram 2
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4500.pt -src drive/MyDrive/Lab3/data/val_fre.txt -output drive/MyDrive/Lab3/data/val_4500_20.txt -gpu 0 -beam_size 20 -seed 531 -block_ngram 2


[2022-03-11 06:49:00,667 INFO] Translating shard 0.
[2022-03-11 06:49:22,714 INFO] PRED AVG SCORE: -0.3432, PRED PPL: 1.4094
[2022-03-11 06:49:26,365 INFO] Translating shard 0.
[2022-03-11 06:50:07,621 INFO] PRED AVG SCORE: -0.3388, PRED PPL: 1.4033
[2022-03-11 06:50:11,329 INFO] Translating shard 0.
[2022-03-11 06:51:09,823 INFO] PRED AVG SCORE: -0.3372, PRED PPL: 1.4010
[2022-03-11 06:51:13,559 INFO] Translating shard 0.
[2022-03-11 06:52:26,839 INFO] PRED AVG SCORE: -0.3371, PRED PPL: 1.4009


In [ ]:
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/val_eng.txt < drive/MyDrive/Lab3/data/val_4500_5.txt
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/val_eng.txt < drive/MyDrive/Lab3/data/val_4500_10.txt
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/val_eng.txt < drive/MyDrive/Lab3/data/val_4500_15.txt
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/val_eng.txt < drive/MyDrive/Lab3/data/val_4500_20.txt

BLEU = 36.45, 63.6/43.1/30.0/21.4 (BP=1.000, ratio=1.010, hyp_len=12287, ref_len=12167)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 36.89, 64.1/43.7/30.4/21.7 (BP=1.000, ratio=1.001, hyp_len=12180, ref_len=12167)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 36.98, 64.3/43.9/30.7/21.9 (BP=0.997, ratio=0

Evaluation on Test set

In [ ]:
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4500.pt -src drive/MyDrive/Lab3/data/test_fre.txt -output drive/MyDrive/Lab3/data/test_4500.txt -gpu 0 -beam_size 20 -seed 531 -block_ngram 2
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/test_eng.txt < drive/MyDrive/Lab3/data/test_4500.txt

[2022-03-11 20:56:48,989 INFO] Translating shard 0.
[2022-03-11 20:57:55,358 INFO] PRED AVG SCORE: -0.3363, PRED PPL: 1.3998
BLEU = 38.55, 65.3/45.3/32.0/23.6 (BP=0.997, ratio=0.997, hyp_len=11841, ref_len=11877)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


BLEU scores for Long and short test sentences

In [ ]:
#long
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4500.pt -src drive/MyDrive/Lab3/data/long_test_fre.txt -output drive/MyDrive/Lab3/data/long_test_4500.txt -gpu 0 -beam_size 20 -seed 531 -block_ngram 2
#short
!onmt_translate -model drive/MyDrive/Lab3/data/run/model_step_4500.pt -src drive/MyDrive/Lab3/data/short_test_fre.txt -output drive/MyDrive/Lab3/data/short_test_4500.txt -gpu 0 -beam_size 20 -seed 531 -block_ngram 2

[2022-03-11 20:39:23,882 INFO] Translating shard 0.
[2022-03-11 20:39:32,834 INFO] PRED AVG SCORE: -0.5017, PRED PPL: 1.6515
[2022-03-11 20:39:36,459 INFO] Translating shard 0.
[2022-03-11 20:39:39,335 INFO] PRED AVG SCORE: -0.2544, PRED PPL: 1.2897


In [ ]:
#Long
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/long_test_eng.txt < drive/MyDrive/Lab3/data/long_test_4500.txt
#Short
!perl  drive/MyDrive/Lab3/multi-bleu.perl drive/MyDrive/Lab3/data/short_test_eng.txt < drive/MyDrive/Lab3/data/short_test_4500.txt

BLEU = 28.93, 60.5/37.8/24.4/16.3 (BP=0.937, ratio=0.938, hyp_len=1342, ref_len=1430)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 37.77, 62.4/45.0/31.7/22.9 (BP=1.000, ratio=1.082, hyp_len=609, ref_len=563)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


Working on transformer

In [ ]:
!onmt_build_vocab -config multi30k-Transform.yml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-11 22:49:03,335 INFO] Counter vocab from 10000 samples.
[2022-03-11 22:49:03,335 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-03-11 22:49:03,344 INFO] corpus_1's transforms: TransformPipe()
[2022-03-11 22:49:03,549 INFO] Counters src:9483
[2022-03-11 22:49:03,549 INFO] Counters tgt:8615


In [6]:
!onmt_train -config multi30k-Transform.yml

[2022-03-11 22:49:07,896 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-11 22:49:07,897 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-11 22:49:07,897 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-11 22:49:07,897 INFO] Parsed 2 corpora from -data.
[2022-03-11 22:49:07,898 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-11 22:49:07,898 INFO] Loading vocab from text file...
[2022-03-11 22:49:07,898 INFO] Loading src vocabulary from drive/MyDrive/Lab3/data/run/example_transform.vocab.src
[2022-03-11 22:49:07,925 INFO] Loaded src vocab has 9483 tokens.
[2022-03-11 22:49:07,930 INFO] Loading tgt vocabulary from drive/MyDrive/Lab3/data/run/example_transform.vocab.tgt
[2022-03-11 22:49:07,951 INFO] Loaded tgt vocab has 8615 tokens.
[2022-03-11 22:49:07,955 INFO] Building fields with vocab in counters...
[2022-03-11 22:49:07,965 INFO]  * tgt vocab size: 8619.


Four models analysed: (1) basic seq2seq model; (2) seq2seq with additive attention ; (3) seq2seq variant with Bidirectional LSTM encoder; and finally (4) Open-NMT seq2seq with MLP attention. 

# Cumulative BLEU-4 score on test set:
- For model (1), **my best model obtains 24.95 cumulative BLEU-4 score with 8 epoch(s).** 
- For model (2), **my best model obtains 41.8 cumulative BLEU-4 score with 4 epoch(s).** 
- For model (3), **my best model obtains 24.55 cumulative BLEU-4 score with 6 epoch(s).** 
- For model (4), **my best model obtains 38.55 cumulative BLEU-4 score with 4500 epoch(s).** 

# Cumulative BLEU-4 score on long sentences of test set:
- For model (1), **my best model obtains 15.19 cumulative BLEU-4.** 
- For model (2), **my best model obtains 30.4 cumulative BLEU-4.**
- For model (3), **my best model obtains 15.91 cumulative BLEU-4.**
- For model (4), **my best model obtains 28.93 cumulative BLEU-4.**

# Cumulative BLEU-4 score on short sentences of test set:
- For model (1), **my best model obtains 32.15 cumulative BLEU-4.** 
- For model (2), **my best model obtains 39.4 cumulative BLEU-4.**
- For model (3), **my best model obtains 31.96 cumulative BLEU-4.**
- For model (4), **my best model obtains 37.77 cumulative BLEU-4.**

# Model Comparison:

1) When compared to the other models, the attention-based model performs the best. The cumulative BLEU-4 score from the attention-based model is 41.8. 
2) With a total BLEU-4 score of 38.55, the openNMT rnn-based model comes in second. 
3) The regular seq2seq model with BLEU score-24.95 outperforms the bidirectional lstm encoder seq2seq model with BLEU score-24.55. 
4) As the length of the phrases is increased or when long sentences are fed into the corresponding models, the BLEU scores decline, and therefore the performance decreases.